<a href="https://colab.research.google.com/github/srirambandi/GAN/blob/master/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when running in colab notebooks, first install library
!pip install import-ai
# upload respective dataset manually from examples directory of the library or download as below
!apt install subversion
!svn checkout https://github.com/srirambandi/ai/trunk/examples/MNIST

In [2]:
import ai
import numpy as np

In [3]:
z_dim = 100
gf_dim = 64
df_dim = 64

In [4]:
ai.manual_seed(2357)

In [5]:
def data_generator(m):
    train_dict = np.load('MNIST/train.npy', allow_pickle=True)
    test_dict = np.load('MNIST/test.npy', allow_pickle=True)
    data = np.concatenate([train_dict.item()['data'], test_dict.item()['data']])
    data = data.transpose(1, 2, 0)   # making data batch-last
    data = data.reshape(1, *data.shape) / 255   # adding channel dimension and normalizing data
    epoch = 0
    
    while True:
        epoch += 1
        for batch in range(int(data.shape[-1] / m)):
            yield data[...,batch * m:(batch + 1) * m], epoch

In [6]:
class Generator(ai.Module):
    def __init__(self):
        self.g_fc = ai.Linear(z_dim, 8*gf_dim * 2 * 2)
        self.g_bn1 = ai.BatchNorm((8*gf_dim, 2, 2))
        self.g_deconv1 = ai.ConvTranspose2d(8*gf_dim, 4*gf_dim, kernel_size=5, stride=2, padding=2, a=1)
        self.g_bn2 = ai.BatchNorm((4*gf_dim, 4, 4))
        self.g_deconv2 = ai.ConvTranspose2d(4*gf_dim, 2*gf_dim, kernel_size=5, stride=2, padding=2, a=0)
        self.g_bn3 = ai.BatchNorm((2*gf_dim, 7, 7))
        self.g_deconv3 = ai.ConvTranspose2d(2*gf_dim, gf_dim, kernel_size=5, stride=2, padding=2, a=1)
        self.g_bn4 = ai.BatchNorm((gf_dim, 14, 14))
        self.g_deconv4 = ai.ConvTranspose2d(gf_dim, 1, kernel_size=5, stride=2, padding=2, a=1)
        
    def forward(self, z):
        o1 = ai.G.reshape(self.g_fc(z), (8*gf_dim, 2, 2))
        o2 = ai.G.relu(self.g_bn1(o1))
        o3 = ai.G.relu(self.g_bn2(self.g_deconv1(o2)))
        o4 = ai.G.relu(self.g_bn3(self.g_deconv2(o3)))
        o5 = ai.G.relu(self.g_bn4(self.g_deconv3(o4)))
        fake_image = ai.G.tanh(self.g_deconv4(o5))
        return fake_image

In [7]:
class Discriminator(ai.Module):
    def __init__(self):
        self.d_conv1 = ai.Conv2d(1, 64, kernel_size=5, stride=2, padding=2)
        self.d_conv2 = ai.Conv2d(64, 2*64, kernel_size=5, stride=2, padding=2)
        self.d_bn1 = ai.BatchNorm((2*64, 7, 7))
        self.d_conv3 = ai.Conv2d(2*64, 3*64, kernel_size=5, stride=2, padding=2)
        self.d_bn2 = ai.BatchNorm((3*64, 4, 4))
        self.d_conv4 = ai.Conv2d(3*64, 4*64, kernel_size=5, stride=2, padding=2)
        self.d_bn3 = ai.BatchNorm((4*64, 2, 2))
        self.d_fc = ai.Linear(1024, 1)
        
    def forward(self, image):
        o1 = ai.G.lrelu(self.d_conv1(image))
        o2 = ai.G.lrelu(self.d_bn1(self.d_conv2(o1)))
        o3 = ai.G.lrelu(self.d_bn2(self.d_conv3(o2)))
        o4 = ai.G.lrelu(self.d_bn3(self.d_conv4(o3)))
        o5 = self.d_fc(o4)
        return ai.G.sigmoid(o5)

In [8]:
generator = Generator()
discriminator = Discriminator()
print(generator)
print(discriminator)

Generator(
  g_fc: Linear(input_features=100, output_features=2048, bias=True)
  g_bn1: BatchNorm((512, 2, 2), axis=-1, momentum=0.9, bias=True)
  g_deconv1: ConvTranspose2d(512, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), a=(1, 1), bias=True)
  g_bn2: BatchNorm((256, 4, 4), axis=-1, momentum=0.9, bias=True)
  g_deconv2: ConvTranspose2d(256, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), a=(0, 0), bias=True)
  g_bn3: BatchNorm((128, 7, 7), axis=-1, momentum=0.9, bias=True)
  g_deconv3: ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), a=(1, 1), bias=True)
  g_bn4: BatchNorm((64, 14, 14), axis=-1, momentum=0.9, bias=True)
  g_deconv4: ConvTranspose2d(64, 1, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), a=(1, 1), bias=True)
)
Discriminator(
  d_conv1: Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=True)
  d_conv2: Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=True)
  d_bn1: BatchNorm((128

In [9]:
lr = 0.0002
beta1 = 0.5
L = ai.Loss(loss_fn='BCELoss')
g_optim = ai.Optimizer(generator.parameters(), optim_fn='Adam', lr=lr, beta1=beta1)
d_optim = ai.Optimizer(discriminator.parameters(), optim_fn='Adam', lr=lr, beta1=beta1)

In [10]:
it, epoch = 0, 0
m = 64   # batch size
n_discriminator = 1   # number of descriminator updates per generator update

# real images data generator
data = data_generator(m)

sample_z = np.random.randn(z_dim, m)
sample_images = None

In [11]:
def sampler(sample_images):
    ai.G.grad_mode = False

    # generate images like real data
    fake_images = generator.forward(sample_z).data
    fake_images = (fake_images + 1.) / 2.

    if sample_images is not None:
        sample_images = np.concatenate([sample_images, fake_images], axis=-1)
    else:
        sample_images = fake_images

    np.save('sample_images.npy', sample_images)
    
    ai.G.grad_mode = True

In [12]:
for it in range(10000):
    
    # freeze generator before optimizing descriminator
    for p in generator.parameters():
        p.eval_grad = False

    # training descriminator to identify real/fake data
    for _ in range(n_discriminator):

        real_images, epoch = data.__next__()
        real_labels = np.ones((1, m))
        if (real_images.shape[-1] != m):
            continue

        real_probs = discriminator.forward(real_images)
        d_loss_real = L.loss(real_probs, real_labels)

        z = np.random.randn(z_dim, m)
        fake_images = generator.forward(z)
        fake_labels =  np.zeros((1, m))

        fake_probs = discriminator.forward(fake_images)
        d_loss_fake = L.loss(fake_probs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        d_loss.grad = np.zeros(d_loss.shape)

        d_loss.backward()
        d_optim.step()
        d_optim.zero_grad()

    # unfreeze generator
    for p in generator.parameters():
        p.eval_grad = True

    # training generator to fool descriminator with fake data
    z = np.random.randn(z_dim, m)
    fake_images = generator.forward(z)
    fake_labels =  np.ones((1, m))

    fake_probs = discriminator.forward(fake_images)
    g_loss = L.loss(fake_probs, fake_labels)

    g_loss.backward()
    g_optim.step()
    g_optim.zero_grad()
    d_optim.zero_grad()

    if it%10 == 0:
        print('Iter: {}, Epoch: {}, d_loss: {}, g_loss: {}'.format(it, epoch, d_loss.data[0, 0], g_loss.data[0, 0]))
    
    if it%100 == 0:
        sampler(sample_images)
        generator.save()
        discriminator.save()

using Adam
using Adam
Iter: 0, Epoch: 1, d_loss: 1.4048446477600869, g_loss: 0.6804074710917262
saving model...
Successfully saved model in Generator.npy
saving model...
Successfully saved model in Discriminator.npy
Iter: 10, Epoch: 1, d_loss: 1.3479563831413852, g_loss: 0.7364326437366828
Iter: 20, Epoch: 1, d_loss: 1.2466618940128118, g_loss: 0.7823281578918218
Iter: 30, Epoch: 1, d_loss: 1.1397869121140072, g_loss: 0.8969086439411424
Iter: 40, Epoch: 1, d_loss: 1.018276474696106, g_loss: 0.9880030129121773
Iter: 50, Epoch: 1, d_loss: 1.059570664449264, g_loss: 1.0494747893989744
Iter: 60, Epoch: 1, d_loss: 0.8187982379145465, g_loss: 1.1898379829550205
Iter: 70, Epoch: 1, d_loss: 0.8689026057899547, g_loss: 1.1944627486030963
Iter: 80, Epoch: 1, d_loss: 0.776076467518586, g_loss: 1.2416419256731888
Iter: 90, Epoch: 1, d_loss: 0.5566131166461864, g_loss: 1.4670736797058295
Iter: 100, Epoch: 1, d_loss: 0.600054149024182, g_loss: 1.4980203464252753
saving model...
Successfully saved mo

/usr/local/lib/python3.6/dist-packages/ai/graph.py:155: RuntimeWarning: divide by zero encountered in log
  out = ai.parameter.Parameter(data=np.log(h.data), graph=self)
/usr/local/lib/python3.6/dist-packages/ai/graph.py:75: RuntimeWarning: invalid value encountered in multiply
  out = ai.parameter.Parameter(data=np.multiply(x.data, y.data), graph=self)
/usr/local/lib/python3.6/dist-packages/ai/graph.py:161: RuntimeWarning: divide by zero encountered in true_divide
  h.grad += np.multiply(out.grad, np.divide(1.0, h.data))
/usr/local/lib/python3.6/dist-packages/ai/graph.py:161: RuntimeWarning: invalid value encountered in multiply
  h.grad += np.multiply(out.grad, np.divide(1.0, h.data))
/usr/local/lib/python3.6/dist-packages/ai/graph.py:546: RuntimeWarning: invalid value encountered in less
  z.grad[z.data < 0] *= alpha
/usr/local/lib/python3.6/dist-packages/ai/graph.py:528: RuntimeWarning: invalid value encountered in less
  z.grad[z.data < 0] = 0


Iter: 990, Epoch: 1, d_loss: nan, g_loss: nan
Iter: 1000, Epoch: 1, d_loss: nan, g_loss: nan
saving model...
Successfully saved model in Generator.npy
saving model...
Successfully saved model in Discriminator.npy
Iter: 1010, Epoch: 1, d_loss: nan, g_loss: nan


KeyboardInterrupt: ignored